<a href="https://colab.research.google.com/github/kgpark88/deeplearning/blob/master/model_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 서빙 서비스 구현하기 
## 자체 구축(Flask, Django, 기타), TensorFlow Serving, TorchServ 중 선택

## 기본 라이브러리 임포트(import)

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## 학습데이터와 검증데이터셋 준비

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

## 모델 구성 및 빌드(build)
- 이미지분류
- 감정분류(BERT사용)
- OTV 선호 장르 예측
- (통신/교통/COVID 등) 발생량 예측
- (에너지 등) 사용량 예측
- 기타

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.1140 - accuracy: 0.7066 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3695 - accuracy: 0.8981 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3154 - accuracy: 0.9100 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2765 - accuracy: 0.9223 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2556 - accuracy: 0.9276 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2367 - accuracy: 0.9321 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2198 - accuracy: 0.9390 - val_loss: 0.1970 - val_accuracy:

In [5]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

## 모델 서빙용 모델 저장

In [6]:
model_version = "001"
model_name = "abc_model"
model_path = os.path.join(model_name, model_version)
model_path

'abc_model/001'

In [7]:
!rm -rf {model_name}

In [8]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: abc_model/001/assets


In [9]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

abc_model/
    001/
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


## 저장된 모델 확인

In [10]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0204 02:14:10.220359 139938635921280 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/p

## 모델 서빙 서비스 구현
### 아래 방법 중 선택
- 자체 구현(Flask, Django)
- TensorFlow Serving
- TorchServ 

예시1) TensorFlow Serving [Docker](https://docs.docker.com/install/)를 설치하고 실행하세요. REST API 서비스 포트는 8888번으로 지정하세요.

```bash
docker pull tensorflow/serving

export ML_PATH=$HOME/ml # or wherever this project is
docker run -it --rm -p 7777:7777 -p 8888:8888 \
   -v "$ML_PATH/abc_model:/models/abc_model" \
   -e MODEL_NAME=abc_model \
   tensorflow/serving
```

예시2) tensorflow_model_server 를 설치하고 실행하세요. REST API 서비스 포트는 8888번으로 지정하세요.

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
!curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update && apt-get install -y tensorflow-model-server
!pip install -q -U tensorflow-serving-api

In [12]:
os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]

In [13]:
os.environ["MODEL_DIR"]

'/content/abc_model'

In [29]:
%%bash --bg
nohup tensorflow_model_server \
     --rest_api_port=8888 \
     --model_name=abc_model \
     --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 2 in a separate thread.


In [30]:
!tail server.log

2021-02-04 03:04:16.691636: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /content/abc_model/001
2021-02-04 03:04:16.694218: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 32197 microseconds.
2021-02-04 03:04:16.694640: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /content/abc_model/001/assets.extra/tf_serving_warmup_requests
2021-02-04 03:04:16.694742: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: abc_model version: 1}
2021-02-04 03:04:16.695637: I tensorflow_serving/model_servers/server.cc:371] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 223] NET_LOG: Couldn't bind to port 8888
[evhttp_server.cc : 63] NET_LOG: Server has not been terminated. For

## REST API로 서비스를 요청(request)하고 응답(resposne) 결과를 출력하세요.
- 요청 데이터 양식 : JSON
```
{
    "signature_name": "serving_default",
    "instances": data,
}
```


In [44]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [45]:
import requests

SERVER_URL = 'http://localhost:8888/v1/models/abc_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [46]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])